In [80]:
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [82]:
#Get credentials from gmail API(jackline.w.gathoni.24@dartmouth.edu)
from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/gmail.modify']


def getService():
    """Shows basic usage of the Gmail API.
    Lists the user's Gmail labels.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('gmail_token.json'):
        creds = Credentials.from_authorized_user_file('gmail_token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('gmail_token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        # Call the Gmail API
        service = build('gmail', 'v1', credentials=creds)
        return service

    except HttpError as error:
        # TODO(developer) - Handle errors from gmail API.
        print(f'An error occurred: {error}')



In [83]:
# Reads incoming emails to check if theyre related to shift-dropping or not and then parse for relevant info(date, time and location).
import base64
service =getService()

def getEmails(service):
    email_details={}
    # to read new messages, get unread specifically
    # results = service.users().messages().list(userId="me", labelIds=["INBOX"], q="from:specific email, is:unread").execute()
    results = service.users().messages().list(maxResults=1, userId='me', labelIds=["INBOX"]).execute()

    # add check for no new messages if you decide to do unread messages only
    messages = results.get('messages')
  
    # messages is a list of dictionaries where each dictionary contains a message id.
  
    # iterate through all the messages
    for msg in messages:
        # Get the message from its id
        txt = service.users().messages().get(userId='me', id=msg['id']).execute()
  
        # Use try-except to avoid any Errors
        try:
            # Get value of 'payload' from dictionary 'txt'
            payload = txt['payload']
            headers = payload['headers']
            # parts = payload.get('parts')[0] #-> could be some source of error
            parts = payload.get('parts')
  
            # Look for Subject and Sender Email in the headers
            for d in headers:
                if d['name'] == 'Subject':
                    subject = d['value']
                if d['name'] == 'From':
                    sender = d['value']
            
                if d['name'] == 'To':
                    recipient = d['value']
  
            # from stacksoiverflow to decode the body into plain text
            for part in parts:
                if part["mimeType"] in ["text/plain"]:
                    data = base64.urlsafe_b64decode(part["body"]["data"]).decode("utf-8")


            # Printing the subject, sender's email and message
            # print("Subject: ", subject)
            # print('\n')
            # print("From: ", sender)
            # print('\n')
            # print("Message: ", data)
            # print('\n')

            email_details["Subject"]=subject
            email_details["Sender"]= sender
            email_details["Message"]= data
            email_details["Recipients"]= recipient
        
            
        except:
            pass
    return email_details
  
  
print("The deets include:" ,getEmails(service))

The deets include: {'Subject': 'Assignment Graded: Exercise 8: Info Extraction and ASR, Accelerated Computational Linguistics-01 (SP23)', 'Sender': '"Accelerated Computational Linguistics-01 (SP23)" <notifications@instructure.com>', 'Message': '\r\nYour assignment Exercise 8: Info Extraction and ASR has been graded.\r\n\r\ngraded: Jun 5 at 2:29pm\r\nscore: 3.75 out of 5.0\r\n\r\n\r\nYou can review the assignment here:\r\nhttps://dartmouth.instructure.com/courses/58305/assignments/409895/submissions/137415\r\n\r\n\r\n\r\n\r\n\r\n\r\n________________________________________\r\n\r\nYou received this email because you are participating in one or more classes using Canvas.  To change or turn off email notifications, visit:\r\nhttps://dartmouth.instructure.com/profile/communication\r\n\r\n', 'Recipients': 'Jackline.W.Gathoni.24@dartmouth.edu'}


In [78]:
import re

# Regular expression for extracting the sender's email from sender's information: .<(.*)>
# reEmail= r".<(.*)>"
# groupEmail = re.search(reEmail, senderInfo, re.IGNORECASE)
# destination = groupEmail
# check that the recipient email is an email other than my specific email

# Add subject to the text incase it has information about shift dropping
reEmail= r".<(.*)>"
emailDetails = getEmails(service)


destination = re.search(reEmail, emailDetails["Sender"], re.IGNORECASE ).group(1)    

recipients= re.search(reEmail, emailDetails["Recipients"], re.IGNORECASE).group(1)
print("other recipient email is: ", recipients)

KeyError: 'Sender'

In [ ]:
#set pm as a boolean
# 12-2am- Strip down am to plain time-> If 12 am change that to 0000
# 12-2pm- Add 12 hours to both numbers and strip down pm
# noon-1pm - Change noon to 12pm 
# 7:45am-10am - Strip down the ams and retain the times
# 12:00am-2:00am 
# 2to3pm- Replace ‘to’ with -
# 4-5 pm - Deal with space? (get the pm add 12 hours for standard time)

#regex for parsing time
#expression results in five groups
#([0-9]*:*[0-9]*)(.*)(-|to)([0-9]*:*[0-9]*)(.*)
#Group1 = start_time
#Group2 = am/pm/ other exceptions like noon, midnight
#Group3 = -/to
#Group4 = end_time 
#Group5 = am/pm/other exceptions like noon, midnight


pm = False


In [45]:
#Sends email responding to the sender if available for the specific time window of the shift dropped

from base64 import urlsafe_b64encode
from email.mime.text import MIMEText
service =getService()

#think about adding recipients field for generic emails like work-place emails
#loop through recipeints in case there's more than one
def build_message(destination, obj, body):
    message = MIMEText(body)

    message['to'] = destination
    message['from'] = "jackline.w.gathoni.24@dartmouth.edu"
    message['subject'] = obj
    message['In-Reply-To']= destination
    message['References']= recipients

    return {'raw': urlsafe_b64encode(message.as_bytes()).decode()}

def send_message(service, destination, obj, body):
    return service.users().messages().send(
      userId="me",
      body=build_message(destination, obj, body)
    ).execute()


# test send email
send_message(service, "ivyjacky2@gmail.com", emailDetails["Subject"], 
            "I can",
)

{'id': '188879cb70c919d4',
 'threadId': '188879cb70c919d4',
 'labelIds': ['SENT']}